# SQL | Intermediate Joins
---
## Concepts
- A __recursive join__ is joining a table to itself.
- We can use the __pipe operator (`||`)__ to concatenate columns.
- __`LIKE`__ statement for partial matches.

In [1]:
# importing libraries and database

import sqlite3
import pandas as pd

conn = sqlite3.connect("chinook.db")

In [2]:
q1 = ''' SELECT * FROM sqlite_master WHERE type='table' '''
pd.read_sql_query(q1, conn)

,type,name,tbl_name,rootpage,sql
0,table,album,album,2,CREATE TABLE [album]\n(\n [album_id] INTEGE...
1,table,artist,artist,3,CREATE TABLE [artist]\n(\n [artist_id] INTE...
2,table,customer,customer,4,CREATE TABLE [customer]\n(\n [customer_id] ...
3,table,employee,employee,5,CREATE TABLE [employee]\n(\n [employee_id] ...
4,table,genre,genre,6,CREATE TABLE [genre]\n(\n [genre_id] INTEGE...
5,table,invoice,invoice,7,CREATE TABLE [invoice]\n(\n [invoice_id] IN...
6,table,invoice_line,invoice_line,8,CREATE TABLE [invoice_line]\n(\n [invoice_l...
7,table,media_type,media_type,9,CREATE TABLE [media_type]\n(\n [media_type_...
8,table,playlist,playlist,10,CREATE TABLE [playlist]\n(\n [playlist_id] ...
9,table,playlist_track,playlist_track,11,CREATE TABLE [playlist_track]\n(\n [playlis...


#### Database schema:
<img src = 'https://s3.amazonaws.com/dq-content/189/chinook-schema.svg' style="width: 600px;">

### Instructions

Write a query that gathers data about the invoice with an __`invoice_id`__ of __`4`__. Include the following columns in order:
- The id of the track, __`track_id`__.
- The name of the track, __`track_name`__.
- The name of media type of the track, __`track_type`__.
- The price that the customer paid for the track, __`unit_price`__.
- The quantity of the track that was purchased, __`quantity`__.

In [3]:
q1 = '''SELECT
    il.track_id,
    t.name track_name,
    mt.name track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
WHERE il.invoice_id = 4; '''
pd.read_sql_query(q1, conn)

,track_id,track_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",Protected AAC audio file,0.99,1
1,2560,Violent Pornography,MPEG audio file,0.99,1
2,3336,War Pigs,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,MPEG audio file,0.99,1
4,1872,Attitude,MPEG audio file,0.99,1
5,748,Dealer,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),MPEG audio file,0.99,1
7,2514,Spoonman,MPEG audio file,0.99,1


### Instructions

1. Add a column containing the artists name to the query from the previous screen.
    - The column should be called __`artist_name`__
    - The column should be placed between __`track_name`__ and __`track_type`__

In [4]:
q1 = '''SELECT
    il.track_id,
    t.name track_name,
    ar.name artist_name,
    mt.name track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON al.artist_id = ar.artist_id
WHERE il.invoice_id = 4; '''
pd.read_sql_query(q1, conn)

,track_id,track_name,artist_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",The King's Singers,Protected AAC audio file,0.99,1
1,2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
2,3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
4,1872,Attitude,Metallica,MPEG audio file,0.99,1
5,748,Dealer,Deep Purple,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
7,2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


### Instructions

1. Write a query that returns the top 5 albums, as calculated by the number of times a track from that album has been purchased. Your query should be sorted from most tracks purchased to least tracks purchased and return the following columns, in order:
- __`album`__, the title of the album
- __`artist`__, the artist who produced the album
- __`tracks_purchased`__ the total number of tracks purchased from that album

In [5]:
q1 = '''SELECT
    ta.album_title album,
    ta.artist_name artist,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name,
                al.title album_title
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 3 DESC LIMIT 5;'''
pd.read_sql_query(q1, conn)

,album,artist,tracks_purchased
0,Are You Experienced?,Jimi Hendrix,187
1,Faceless,Godsmack,96
2,Mezmerize,System Of A Down,93
3,Get Born,JET,90
4,The Doors,The Doors,83


### Instructions
1. Write a query that returns information about each employee and their supervisor.
    - The report should include employees even if they do not report to another employee.
    - The report should be sorted alphabetically by the employee_name column.
    - Your query should return the following columns, in order:
        - __`employee_name`__ - containing the __`first_name`__ and __`last_name`__ columns separated by a space, eg __`Luke Skywalker`__
        - __`employee_title`__ - the title of that employee
        - __`supervisor_name`__ - the first and last name of the person the employee reports to, in the same format as __`employee_name`__
        - __`supervisor_title`__ - the title of the person the employee reports to

In [6]:
q1 = '''SELECT
    e1.first_name || " " || e1.last_name employee_name,
    e1.title employee_title,
    e2.first_name || " " || e2.last_name supervisor_name, 
    e2.title supervisor_title
FROM employee e1
LEFT JOIN employee e2 on e1.reports_to = e2.employee_id
ORDER BY 1;'''
pd.read_sql_query(q1, conn)

,employee_name,employee_title,supervisor_name,supervisor_title
0,Andrew Adams,General Manager,None,None
1,Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
2,Laura Callahan,IT Staff,Michael Mitchell,IT Manager
3,Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager
4,Michael Mitchell,IT Manager,Andrew Adams,General Manager
5,Nancy Edwards,Sales Manager,Andrew Adams,General Manager
6,Robert King,IT Staff,Michael Mitchell,IT Manager
7,Steve Johnson,Sales Support Agent,Nancy Edwards,Sales Manager


### Instructions
You have just returned from lunch to see another phone message on your desk: __`"Call Belle."`__

1. Write a query that finds the contact details of a customer with a __`first_name`__ containing __`Belle`__ from the database. Your query should include the following columns, in order:
    - __`first_name`__
    - __`last_name`__
    - __`phone`__

In [7]:
q1 = '''SELECT
    first_name,
    last_name,
    phone
FROM customer
WHERE first_name LIKE "%belle%";'''
pd.read_sql_query(q1, conn)

,first_name,last_name,phone
0,Isabelle,Mercier,+33 03 80 73 66 99


### Instructions

1. Write a query that summarizes the purchases of each customer. For the purposes of this exercise, we do not have any two customers with the same name.
2. Your query should include the following columns, in order:
    - __`customer_name`__ - containing the first_name and last_name columns separated by a space, eg Luke Skywalker.
    - __`number_of_purchases`__, counting the number of purchases made by each customer.
    - __`total_spent`__ - the total sum of money spent by each customer.
    - __`customer_category`__ - a column that categorizes the customer based on their total purchases. The column should contain the following values:
    - __`small spender`__ - If the customer's total purchases are less than `$40`.
    - __`big spender`__ - If the customer's total purchases are greater than `$100`.
    - __`regular`__ - If the customer's total purchases are between `$40` and `$100` (inclusive).
Order your results by the customer_name column.

In [8]:
q1 = '''SELECT
    c.first_name || " " || c.last_name customer_name,
    COUNT(*) number_of_purchases,
    SUM(i.total) total_spent, 
    CASE
        WHEN SUM(i.total) < 40 THEN 'small spender'
        WHEN SUM(i.total) > 100 THEN 'big spender'
        ELSE 'regular'
        END
        AS customer_category
FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY c.customer_id
ORDER BY 1
LIMIT 5'''
pd.read_sql_query(q1, conn)

,customer_name,number_of_purchases,total_spent,customer_category
0,Aaron Mitchell,8,70.29,regular
1,Alexandre Rocha,10,69.30,regular
2,Astrid Gruber,9,69.30,regular
3,Bjørn Hansen,9,72.27,regular
4,Camille Bernard,9,79.20,regular


In [9]:
#Closing a sqlite3 connection
conn.close()